In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil

shutil.copy('/content/drive/MyDrive/gtFine_trainvaltest.zip', '/content/')
shutil.copy('/content/drive/MyDrive/leftImg8bit_trainvaltest.zip', '/content/')

print("Zip files have been copied to the '/content/' directory.")


In [ ]:
import zipfile

# Extract the zip files
with zipfile.ZipFile('/content/gtFine_trainvaltest.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/gtFine_trainvaltest/')

with zipfile.ZipFile('/content/leftImg8bit_trainvaltest.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/leftImg8bit_trainvaltest/')

print("Zip files have been successfully extracted inside the '/content/' directory.")


In [ ]:
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.amp import autocast, GradScaler
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import shutil

from google.colab import drive
drive.mount('/content/drive')

# ResNet34 + UNet Model
class ResNetUNet(nn.Module):
    def __init__(self, n_classes):
        super(ResNetUNet, self).__init__()
        base_model = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
        base_layers = list(base_model.children())
        self.layer0 = nn.Sequential(*base_layers[:3])
        self.layer1 = nn.Sequential(*base_layers[3:5])
        self.layer2 = base_model.layer2
        self.layer3 = base_model.layer3
        self.layer4 = base_model.layer4
        self.upconv4 = self._upsample(512, 256)
        self.upconv3 = self._upsample(256, 128)
        self.upconv2 = self._upsample(128, 64)
        self.upconv1 = self._upsample(64, 64)
        self.conv_last = nn.Conv2d(64, n_classes, kernel_size=1)

    def _upsample(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x0 = self.layer0(x)
        x1 = self.layer1(x0)
        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        x4 = self.layer4(x3)
        x = self.upconv4(x4)
        x = self.upconv3(x)
        x = self.upconv2(x)
        x = self.upconv1(x)
        x = nn.functional.interpolate(x, scale_factor=2, mode='bilinear', align_corners=True)
        return self.conv_last(x)

# Dataset (Only vehicle classes)
class CityscapesVehicleDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_paths = sorted(glob.glob(os.path.join(image_dir, '**', '*.png'), recursive=True))
        self.label_paths = sorted(glob.glob(os.path.join(label_dir, '**', '*_labelIds.png'), recursive=True))
        self.image_transform = transform
        self.label_resize = transforms.Resize((224, 224), interpolation=Image.NEAREST)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = Image.open(self.label_paths[idx]).convert("L")
        label = self.label_resize(label)
        if self.image_transform:
            image = self.image_transform(image)
        label = np.array(label)
        label = ((label == 26) | (label == 27) | (label == 28) | (label == 31)).astype(np.float32)
        label = torch.from_numpy(label).unsqueeze(0)
        return image, label

# Transforms & Paths
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

IMAGE_ROOT = '/content/leftImg8bit_trainvaltest/leftImg8bit/train'
LABEL_ROOT = '/content/gtFine_trainvaltest/gtFine/train'

dataset = CityscapesVehicleDataset(IMAGE_ROOT, LABEL_ROOT, transform)
loader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetUNet(n_classes=1).to(device)

loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([20.0]).to(device))
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
scaler = GradScaler()

# Training Loop
EPOCHS = 50
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for i, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast(device_type="cuda"):
            outputs = model(images)
            loss = loss_fn(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()

    scheduler.step()
    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {avg_loss:.4f}")

    # Debug + Metrics
    model.eval()
    with torch.no_grad():
        test_image, test_label = dataset[0]
        test_image = test_image.unsqueeze(0).to(device)
        test_label = test_label.unsqueeze(0).to(device)
        pred_logits = model(test_image)
        pred = torch.sigmoid(pred_logits)
        pred_np = pred[0, 0].cpu().numpy()
        label_np = test_label[0, 0].cpu().numpy()
        pred_bin = (pred > 0.2).float()

        if test_label.sum() == 0:
            print("Warning: No vehicle pixels in this batch.")
        else:
            print(f"[Epoch {epoch+1}] Mask Mean: {label_np.mean():.4f} | Pred Mean: {pred_np.mean():.4f} | Max: {pred_np.max():.4f} | Min: {pred_np.min():.4f} | Std: {pred_np.std():.4f}")
            intersection = (pred_bin * test_label).sum()
            union = pred_bin.sum() + test_label.sum()
            iou = (intersection / (union - intersection + 1e-6)).item()
            dice = (2 * intersection / (union + 1e-6)).item()
            print(f"IOU: {iou:.4f} | Dice: {dice:.4f}")

        plt.imsave(f"/content/debug_mask_ep{epoch+1}.png", label_np * 255, cmap="gray")
        plt.imsave(f"/content/debug_pred_ep{epoch+1}.png", (pred_np > 0.2).astype(np.uint8) * 255, cmap="gray")
        plt.hist(pred_np.flatten(), bins=50)
        plt.title(f"Prediction Histogram (Epoch {epoch+1})")
        plt.savefig(f"/content/debug_pred_hist_ep{epoch+1}.png")
        plt.close()

    if (epoch + 1) % 10 == 0:
        weight_path = f"/content/final_resnet_unet_epoch_{epoch+1}.pth"
        torch.save(model.state_dict(), weight_path)
        shutil.copy(weight_path, f"/content/drive/MyDrive/final_resnet_unet_epoch_{epoch+1}.pth")
        print(f"Saved to Drive: final_resnet_unet_epoch_{epoch+1}.pth")

# Save final model
torch.save(model.state_dict(), "/content/final_resnet_unet_model.pth")
shutil.copy("/content/final_resnet_unet_model.pth", "/content/drive/MyDrive/final_resnet_unet_model.pth")
print("Final model saved.")
